In [1]:
BITSTREAM_PATH = '../overlay/demo.bit'
BUFFER_SIZE_W = 16

In [ ]:
import os
import ipywidgets as widgets
from pynq import allocate
from IPython.display import display
from axi.driver import SimpleAxiMasterDriver

In [3]:
if not os.path.exists(BITSTREAM_PATH):
    raise FileNotFoundError(f'Bitstream not found at {BITSTREAM_PATH}. Run \'make bitstream\'.')

driver = SimpleAxiMasterDriver(BITSTREAM_PATH)

In [6]:
buffer = allocate(shape=(BUFFER_SIZE_W,), dtype='u4')
buffer_addr = buffer.physical_address

buffer[:] = [0] * BUFFER_SIZE_W
buffer.flush()

In [ ]:
def hex_dump(buffer, num_bytes=64, row_size=8):
    buffer.invalidate()
    
    lines = []
    header = f"{'Offset':<8} | {'00 01 02 03 04 05 06 07':<23}"
    lines.append(header)
    lines.append("-" * len(header))
    
    byte_list = []
    limit = min(num_bytes // 4, len(buffer))
    for i in range(limit):
        val = buffer[i]
        byte_list.extend([val & 0xFF, (val >> 8) & 0xFF, (val >> 16) & 0xFF, (val >> 24) & 0xFF])

    for i in range(0, len(byte_list), row_size):
        chunk = byte_list[i:i+row_size]
        hex_strs = [f"{b:02X}" for b in chunk]
        row_str = " ".join(hex_strs)
        lines.append(f"+{hex(i):<7} | {row_str}")
        
    return "\n".join(lines)

In [ ]:
# ----------------------------------------------------------------------
# UI Layout and Controls
# ----------------------------------------------------------------------

# -- Inputs --
w_addr = widgets.Text(value=hex(buffer.physical_address), description="Addr:", layout={'width': '180px'})
w_data = widgets.Text(value="0x11223344", description="Data:", layout={'width': '180px'})
w_size = widgets.Dropdown(
    options=[('Byte (8b)', 0), ('Half (16b)', 1), ('Word (32b)', 2), ('DWord (64b)', 3)],
    value=2, description="Size:", layout={'width': '180px'}
)

# -- PL Controls --
btn_pl_write = widgets.Button(description="Write", button_style='warning', icon='microchip')
btn_pl_read  = widgets.Button(description="Read", button_style='success', icon='microchip')
btn_pl_stat  = widgets.Button(description="Get Status", button_style='info', icon='question')
btn_pl_clear = widgets.Button(description="Clear Status", button_style='danger', icon='eraser')
btn_pl_reset = widgets.Button(description="Reset", button_style='danger', icon='power-off')

# -- Python (CPU) Controls --
w_cpu_val = widgets.Text(value="0xAABBCCDD", description="CPU Val:", layout={'width': '180px'})
w_cpu_idx = widgets.BoundedIntText(value=0, min=0, max=BUFFER_SIZE_W-1, description="Word Idx:", layout={'width': '150px'})
btn_cpu_write = widgets.Button(description="CPU Write", button_style='primary', icon='keyboard')
btn_cpu_zero  = widgets.Button(description="Zero Buffer", button_style='', icon='trash')

# -- Outputs --
out_buffer = widgets.Output(layout={'border': '1px solid #444', 'padding': '10px', 'font_family': 'monospace'})
out_log    = widgets.Output(layout={'border': '1px solid #ccc', 'height': '150px', 'overflow_y': 'scroll'})

# ----------------------------------------------------------------------
# Callbacks
# ----------------------------------------------------------------------

def refresh_view():
    out_buffer.clear_output()
    with out_buffer:
        print(f"Buffer base address: {hex(buffer.physical_address)}")
        print(hex_dump(buffer, num_bytes=64))

def log(msg):
    with out_log:
        print(msg)

def on_pl_write(b):
    try:
        addr, data, size = int(w_addr.value, 16), int(w_data.value, 16), w_size.value
        log(f"WRITE -> Addr:{hex(addr)} Data:{hex(data)} Size:{size}")
        res = driver.write(size, addr, data)
        log(f"  Stat: {res.status.value.upper()}")
        log(f"  Latency: {res.latency} cycles") 
        refresh_view()
    except Exception as e: log(f"ERR: {e}")

def on_pl_read(b):
    try:
        addr, size = int(w_addr.value, 16), w_size.value
        log(f"READ  <- Addr:{hex(addr)} Size:{size}")
        res = driver.read(size, addr)
        log(f"  Data: {hex(res.data)}")
        log(f"  Stat: {res.status.value.upper()}")
        log(f"  Latency: {res.latency}")
    except Exception as e: log(f"ERR: {e}")

def on_pl_stat(b):
    s = driver.status
    log(f"Current status register: {bin(s)} (Wait={(s>>0)&1}, Done={(s>>1)&1}, Err={(s>>2)&1})")

def on_pl_clear(b):
    driver.clear()
    log("Status cleared.")

def on_pl_reset(b):
    log("Resetting...")
    driver.hard_reset()
    log("Reset complete.")

def on_cpu_write(b):
    try:
        val, idx = int(w_cpu_val.value, 16), w_cpu_idx.value
        buffer[idx] = val
        buffer.flush()
        log(f"CPU WRITE -> Buffer[{idx}] = {hex(val)}")
        refresh_view()
    except Exception as e: log(f"ERR: {e}")

def on_cpu_zero(b):
    for i in range(len(buffer)): buffer[i] = 0
    buffer.flush()
    log("Buffer zeroed.")
    refresh_view()

# Link
btn_pl_write.on_click(on_pl_write)
btn_pl_read.on_click(on_pl_read)
btn_pl_stat.on_click(on_pl_stat)
btn_pl_clear.on_click(on_pl_clear)
btn_pl_reset.on_click(on_pl_reset)
btn_cpu_write.on_click(on_cpu_write)
btn_cpu_zero.on_click(on_cpu_zero)

# ----------------------------------------------------------------------
# Layout
# ----------------------------------------------------------------------
refresh_view()

ui = widgets.VBox([
    widgets.HTML("<h3>AXI Master Dashboard</h3>"),
    widgets.HBox([widgets.Label("PL Control:", layout={'width':'80px'}), w_addr, w_data, w_size]),
    widgets.HBox([widgets.Label("", layout={'width':'80px'}), btn_pl_write, btn_pl_read, btn_pl_stat, btn_pl_clear, btn_pl_reset]),
    widgets.HTML("<hr>"),
    widgets.HBox([widgets.Label("CPU Control:", layout={'width':'80px'}), w_cpu_idx, w_cpu_val, btn_cpu_write, btn_cpu_zero]),
    widgets.HTML("<hr>"),
    widgets.Label("Memory View (Little Endian):"),
    out_buffer,
    widgets.Label("Log:"),
    out_log
])

display(ui)